# Probabilistic Latent Semantic Analysis

## Introduction

Probabilisic Latent Semantic Analysos (pLSA) is an algorithm from the family of topic models. 
Its main goal is to model cooccurrence of 
information under a probabilistic
framework in order to discover the underlying
semantic structure of the data. In this sense, words that cooccur often have high probability of belonging to the same theme. 

Although the model has been applied in several domains (text mining, information retrieval, vision, etc.) we focus today in a text mining application that builds on what we have previously discussed in the "Data Mining" course, especially in the last session where we covered the vector space model. 

We will be speaking for three types of variables: 
* Documents: $d \in D = \{d_1, \dots, d_N \}$ observed variables. Let N be their number, defined by the size of our given corpus.
* Words: $w \in W = \{w_1, \dots , w_M\}$ observed variables. Let M be the number of distinct words from the corpus.
* Topics: $z \in Z = \{z_1, \dots , z_K\}$ latent (or hidden) variables. Their number, K, has to be specified a priori.

The figure below shows the generative story of pLSA.  
![The generative story of pLSA](./plsa.png)

This figure reds like this: 
* First we select a document dn with probability $P(d)$.
* For each word $w_i, i \in \{1, · · · , N_w \}$ in the document $d_n$:
     + Select a topic $z_i$  from a multinomial conditioned on the given document with probability P(z|dn).
     + Select a word $w_i$  from a multinomial conditioned on the previous chosen topic with probability P(w|zi).


## Inference
We refer to inference as the process of calculating the unknown parameters of our model, while observing the observed ones. In our case we observe the words of the documents and we want to infer the probabilities. We need to decide a priori the number of topics that exist in the corpus. 


### Step 1: from raw text to counts

We start by creating the document-term matrix for the corpus.
* pre-process the text by cleaning it,
* extracting the vocabulary
* creating the matrix `document_term_matrix`, that has dimensions: |number_of_documents|x|vocabulary_size|

You are encouraged to used the functions you wrote in the previous session, or the ones that have been made available in the Wiki. 

In [13]:
from string import punctuation
import re

documents = open('../session3/wikipedia_data.txt').read().splitlines() # Load documents
documents = documents[:50] # Take only 50 of them 

def get_vocabulary(text):
    """Calculates simple vocabulary counts. Returns:
    word2id: dictionary where each word is associated with an id
    word_counts: dictionary where the counts of the word occurences are saved
    Prep-processing steps: only lower-casing is applied. TBD: add more elaborate pre-processing steps.
    """
    word2id, word_counts, cnt={}, {}, 0
    for document in text:
        document = document.lower()
        words = document.split()
        for word in words:
            try:
                word2id[word]
                word_counts[word] += 1
            except:
                word2id[word] = cnt
                cnt +=1
                word_counts[word] = 1
    return word2id, word_counts

stopwords = set( open('../session3/stop_words.txt').read().splitlines() )

def clean_text(text, stop_words=None, white_space_punctuation=True, remove_non_letters=True, lower=True):
    """
    Cleans the text of a document.
    Input text: type string: the contents of a document
    Returns: a string, with words separated by white space
    Optional:
        stop_words: if True, filter stop words
        white_space_punctuation: append before and after punctuation white space
        remove_non_letters: remove every word that does not consist of characters
        lower: if True, lowercase the text
    """

    if lower:
        text = text.lower()
    if white_space_punctuation:
        for punct_symbol in punctuation:
            try:
                text = text.replace(punct_symbol, " %s "%punct_symbol)
            except:
                pass
    if remove_non_letters:
        text = re.sub("[^a-zA-Z]", " ", text)
    if stop_words != None:
        text = [x for x in text.split() if x not in stop_words]
        text = " ".join(text)
        
    return text

documents = [clean_text(instance, stop_words=stopwords) for instance in documents]
print("There are %d documents in the file."%len(documents) )

There are 50 documents in the file.


In [14]:
from scipy import sparse
def term_frequency_vectorizer(text, vocabulary=None):
    """
    Returns the tf representation of a corpus in sparse format
    Input: text: cleaned text
    Optional: vocabulary: the number of distinct words. If it is given, use only these terms 
    and ignore out-of-vocabulary terms. Otherwise, generate the vocabulary.
    """

    # do a first pass to generate vocabulary
    if vocabulary is None:
        vocabulary = get_vocabulary(text)[0]
        
        
    row = []
    col = []
    values = []
                
    for doc_id, instance in enumerate(text):
        words = instance.split()
        doc_words = {}
        for word in words:
            try:
                doc_words[vocabulary[word]] +=1
            except:
                doc_words[vocabulary[word]] = 1
        col.extend(doc_words.keys())
        values.extend(doc_words.values())
        row.extend([doc_id]*len(doc_words))
    print len(row), len(col), len(values)
    return vocabulary, sparse.csr_matrix( (values, (row, col)), shape=(len(text), len(vocabulary))).todense()



In [12]:
vocabulary, document_term_matrix = term_frequency_vectorizer(documents)

3331 3331 3331


By this point you have constructed the `document_term_matrix`.

We now need to move forward to with the counter matrices we use to store the probabilities.
We need to initialize and populate: 
* `document_topic_prob`: shape: |documents| x |topics|
* `topic_word_probability`: shape |topics| x |vocabulary|
* `topic_prob`: shape |documents| x |vocabulary| x |topics|

In [4]:
import numpy as np
number_of_documents = len(documents)
number_of_topics = 10
vocabulary_size = term_document_matrix.shape[1]
topic_prob = np.zeros([number_of_documents, term_document_matrix.shape[1], number_of_topics], dtype=np.float) # P(d |w,z)

In [5]:
def normalize(vector):
    sum_of_elements = sum(vector)
    if not np.all(vector >= 0):
        print vector
    return vector / sum_of_elements 
    


In [6]:
document_topic_prob = np.random.random(size = (number_of_documents, number_of_topics)) # P(z | d)
topic_word_prob = np.random.random(size = (number_of_topics, term_document_matrix.shape[1] )) # P(w | z)


for d_index in range(len(documents)):
    document_topic_prob[d_index] = normalize(document_topic_prob[d_index]) # normalize for each document

for z in range(number_of_topics):
    topic_word_prob[z] =  normalize(topic_word_prob[z]) # normalize for each topic


### E-step 
For the E-Step of the process one needs to calculate the following probability: 

* $p(z_k | d_i, w_j) = \frac{p(w_j|z_k) p(z_k|d_i)}{\sum\limits_{l=1}^K p(w_j|z_k)  p(z_k|d_i)}$

This can be calculated by updating the 3-d matrix while iterating over the documents and words.

In [21]:
print "E step:",
for docId, document in enumerate(documents):
    for w_index in range(vocabulary_size):
        prob = document_topic_prob[docId, :] * topic_word_prob[:, w_index]
        prob = normalize(prob)
        topic_prob[docId, w_index] = prob
print "Done!"   

E step: Done!


### M-step

The step involves two updates: 
* $p(w_j| z_k) = \frac{\sum\limits_{i=1}^{N} n(d_i,w_j) p(z_k|d_i,w_j)}{\sum\limits_{m=1}^{M} \sum\limits_{n=1}^{N} n(d_n, w_m) p(z_k, |d_n, w_m)}$
* $ p(z_k|d_i) = \frac{\sum\limits_{j=1}^{M} n(d_i, w_j)p(z_k|d_i, w_j) }{n(d_i)}   $


To perform the first update `topic_word_prob` you need to iterate through topics, words and documents:


`for each topic:
    for each word:
        for each document:
            count how many times the word appears in the doc
            s += count * topic_prob[doc, word, topic]
        topic_word_prob[topic, word] = s
    normalize topic_word_prob[z]`
    
Similarly for updating p(z|d):

`for each document:
    for each topic:
        s = 0
        for each word in the vocabulary:
            count how many times the word appears in the doc
            s += count * topic_prob[doc, word, topic]
        document_topic_prob[doc, topic] = s
    normalize document_topic_prob[d_index]`

In [22]:
# update p(w|z)
print "M step:",
for z in range(number_of_topics):
    for w_index in range(vocabulary_size):
        s = 0
        for d_index in range(number_of_documents):
            count = term_document_matrix[d_index, w_index]
            s = s + count * topic_prob[d_index, w_index, z]
        topic_word_prob[z, w_index] = s
    topic_word_prob[z] = normalize(topic_word_prob[z])

# update p(z|d)
for d_index in range(len(documents)):
    for z in range(number_of_topics):
        s = 0
        for w_index in range(vocabulary_size):
            count = term_document_matrix[d_index, w_index]
            s = s + count * topic_prob[d_index, w_index, z]
        document_topic_prob[d_index][z] = s
    document_topic_prob[d_index] = normalize(document_topic_prob[d_index])

print 'Done!'

M step: Done!


Now that you have the E-step, the M-step you can iterate over these two and visualize the words with the highest probability for each topic. 

In [23]:
# Visualize the top words for a topic..
inverse_vocabulary = {key:val for val,key in vocabulary.iteritems()}
indexes = np.argsort(topic_word_prob[4])
print [ inverse_vocabulary[i] for i in indexes[-20:] ]

['group', 'may', 'u', 'released', 'following', 'known', 'early', 'also', 'first', 'band', 'found', 'march', 'rock', 'company', 'series', 'album', 'new', 'fillmore', 'american', 'trombone']
